In [18]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import LSTM, Dense, Flatten, Input, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2

In [19]:
# Load the pre-trained model
pretrained_model = load_model('gender_age_emotion_model.h5')

In [20]:
# Extract VGG16 base from the pre-trained model
vgg16_base = pretrained_model.get_layer('vgg16')
vgg16_base.trainable = True

ValueError: No such layer: vgg16. Existing layers are: ['conv2d_21', 'activation_30', 'batch_normalization_27', 'conv2d_22', 'activation_31', 'batch_normalization_28', 'max_pooling2d_11', 'dropout_17', 'conv2d_23', 'activation_32', 'batch_normalization_29', 'conv2d_24', 'activation_33', 'batch_normalization_30', 'max_pooling2d_12', 'dropout_18', 'conv2d_25', 'activation_34', 'batch_normalization_31', 'conv2d_26', 'activation_35', 'batch_normalization_32', 'max_pooling2d_13', 'dropout_19', 'conv2d_27', 'activation_36', 'batch_normalization_33', 'conv2d_28', 'activation_37', 'batch_normalization_34', 'max_pooling2d_14', 'dropout_20', 'flatten_4', 'dense_10', 'activation_38', 'batch_normalization_35', 'dropout_21', 'dense_11', 'activation_39', 'batch_normalization_36', 'dropout_22', 'dense_12', 'activation_40'].

In [12]:
# Add custom layers to integrate with LSTM
flatten = Flatten()(vgg16_base.output)
fc1 = Dense(256, activation="relu")(flatten)  # Reduce to match LSTM input

time_distributed = TimeDistributed(Dense(256, activation="relu"))  # TimeDistributed layer for LSTM compatibility
lstm_input = Input(shape=(10, 256))
lstm_layer = LSTM(128, return_sequences=False, dropout=0.3, recurrent_dropout=0.3)(lstm_input)
output_layer = Dense(5, activation="softmax")(lstm_layer)  # Adjusted for multiple emotions

In [13]:
# Define the complete model
smart_model = Model(inputs=[vgg16_base.input, lstm_input], outputs=output_layer)
smart_model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

In [14]:
# Function to preprocess image
def preprocess_image(frame):
    resized_frame = cv2.resize(frame, (224, 224))
    normalized_frame = resized_frame / 255.0  # Normalize to [0,1]
    return np.expand_dims(normalized_frame, axis=0)

In [15]:
# Function to map predictions to labels
def map_predictions(predictions):
    gender = "Male" if predictions[0] > 0.5 else "Female"
    age = int(predictions[1] * 100)  # Age approximation based on prediction
    emotions = ["Happy", "Sad", "Angry", "Neutral", "Surprised"]
    emotion = emotions[np.argmax(predictions[2:])]
    return gender, age, emotion

In [16]:
# Function to smooth predictions
def smooth_predictions(history, new_prediction, alpha=0.8):
    return alpha * np.array(history) + (1 - alpha) * np.array(new_prediction)

In [17]:
# Function to capture webcam feed and make predictions
def capture_and_predict():
    cap = cv2.VideoCapture(0)

    # To store sequences for LSTM
    sequence = []
    max_sequence_length = 10
    prediction_history = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess frame
        processed_frame = preprocess_image(frame)
        vgg16_features = vgg16_base.predict(processed_frame)
        reduced_features = fc1(vgg16_features)  # Reduce features to match LSTM input

        # Maintain sequence length for LSTM
        sequence.append(reduced_features.flatten())
        if len(sequence) > max_sequence_length:
            sequence.pop(0)

        if len(sequence) == max_sequence_length:
            # Convert sequence to proper LSTM input shape
            lstm_input_data = np.expand_dims(sequence, axis=0)

            # Predict using the smart model
            raw_predictions = smart_model.predict([processed_frame, lstm_input_data])[0]

            # Smooth predictions for stability
            if prediction_history is None:
                prediction_history = raw_predictions
            else:
                prediction_history = smooth_predictions(prediction_history, raw_predictions)

            # Map predictions to labels
            gender, age, emotion = map_predictions(prediction_history)

            # Overlay predictions on the frame
            cv2.putText(frame, f"Gender: {gender}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Age: {age}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Emotion: {emotion}", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display webcam feed
        cv2.imshow("Webcam Feed", frame)

        # Break loop on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Start the webcam prediction
capture_and_predict()

1/1 [==============================] - 0s 300ms/step


TypeError: 'KerasTensor' object is not callable